## Thành viên nhóm:
1. Nguyễn Đặng Văn Cảnh, MSSV: 23110135
2. Phạm Ngọc Hào, MSSV: 23110146
3. Võ Đoàn Nguyên Lộc, MSSV: 23110098

## 1.Tìm hiểu về đề tài:

### 1.1. Giới thiệu về đề tài:

- **Tên cuộc thi:** Recruit Restaurant Visitor Forecasting
- **Mô tả:** 
    - Việc dự đoán sẽ dựa vào các thông tin về vị trí (kinh độ, vĩ độ) của nhà hàng, các ngày lễ trong năm, thông tin dự báo thời tiết ngày hôm đó và dữ liệu đặt chỗ thông qua trang web đặt chỗ của nhà hàng và dữ liệu khách đã đến trong cơ sở dữ liệu của nhà hàng.
    - **Output:** số lượng khách đến nhà hàng
- **Link kaggle:** https://www.kaggle.com/competitions/recruit-restaurant-visitor-forecasting/overview
- **Cách chấm điểm:** Thông qua Root Mean Squared Logarithmic Error (RMSLE):
        $$\sqrt{\frac{1}{n}\sum^N_{i = 1}(log(p_i + 1) - log(a_i + 1))^2}$$
        với n là tổng số quan sát,
            $p_i$ là số khách hàng mình dự đoán,
            $a_i$ là số khách hàng thực tế


### 1.2. Ý tưởng để giải quyết bài toán:
- Giải quyết về vấn đề dữ liệu:
    - Dữ liệu cửa hàng
        - Số lượng khách hàng đến cửa hàng trong quá khứ: `air_visit_data.csv`
        - Thời gian trong dataset:  `date_info.csv`
        - Thông tin cửa hàng đính kèm với với trạm thời tiết gần đó: `air_store_info`
        - chuẩn bị tập test `submission.csv`
    - Dữ liệu thời tiết
        - Dữ liệu thời tiết của 1663 trạm
- Tiền xử lí dữ liệu
    - Xử lí giá trị ngoại lai
    - Tính trọng số theo cấp số nhân cho số lượng khác hàng, cùng với một số đặc trưng khác
- Huấn luyện mô hình
    - Mã hóa dữ liệu
    - Chia dữ liệu train, test
    - Huấn luyện mô hình
    - Hiển thị kết quả

## 2. Giải quyết dữ liệu

### Những thư viện cần thiết

In [126]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [127]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm import tqdm
from sklearn.model_selection import KFold
import warnings

### 2.1. Dữ liệu nhà hàng

#### 2.1.1. File `air_visit_data.csv`

In [128]:
path1 = 'data/air_visit_data.csv'
air_visit = pd.read_csv(path1)
air_visit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252108 entries, 0 to 252107
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   air_store_id  252108 non-null  object
 1   visit_date    252108 non-null  object
 2   visitors      252108 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 5.8+ MB


- Thông qua việc đọc dữ liệu ta thấy rằng: file chứ các cột `air_store_id`, `visit_date`, `visitors`
- Ý nghĩa của các cột dữ liệu: 
    - **Với cột dữ liệu `air_store_id`:**
        - Đây là cột dữ liệu cung cấp id của cửa hàng, là một dãy kí tự có độ dài 20
        - Trạng thái: cột chứa 252108 dòng dữ liệu, và không trống
        - Kiểu dữ liệu: `air_store_id` có kiểu dữ liệu là object
    - **Với cột dữ liệu `visit_date`:**
        - Đây là cột dữ liệu chứa thời gian có khách của cửa hàng, chứa ngày tháng năm
        - Trạng thái: cột chứa 252108 dòng dữ liệu, và không trống
        - Kiểu dữ liệu: `visit_date` có kiểu dữ liệu là object
    - **Với cột dữ liệu `visitors`:**
        - Đây là cột dữ liệu mà nhóm phải dự đoán để đưa ra kết quả, là các số nguyên dương
        - Trang thái: cột chứa 252180 dòng dữ liệu, và không trống
        - Kiểu dữ liệu: `visitors` có kiểu dữ liệu là int64

In [129]:
air_visit.head(10)

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6
5,air_ba937bf13d40fb24,2016-01-19,9
6,air_ba937bf13d40fb24,2016-01-20,31
7,air_ba937bf13d40fb24,2016-01-21,21
8,air_ba937bf13d40fb24,2016-01-22,18
9,air_ba937bf13d40fb24,2016-01-23,26


- **Ý tưởng để xử lí file `visit_data.csv`:** 
    - Chuyển dữ liệu `visit_date` kiểu object sang kiểu thời gian chuẩn: nhóm sử dụng to_datetime() để làm việc đó
    - Làm liền mạch chuỗi dữ liệu thời gian: ở bảng dữ liệu phía trên, chúng ta có thể thấy rằng dữ liệu chỉ hiển thị những ngày có khách, vì vậy nhóm đã phải dùng lệnh `resample('1d')` để có thể tạo ra một chuỗi dữ liệu thời gian liên tục, các ngày thiểu - không có khách vẫn được thêm vào
    - Định dạng thời gian theo kiểu YYYY-mm-dd: bằng dt.strftime(%Y-%m-%d)
    - Thay dữ liệu trống: Vốn dĩ những ngày được thêm vào đều không có giá trị, nên nhóm quyết định thay thế bằng 0
    - Thêm nhãn nhận biết: nhóm đã thêm nhãn `was_nil`. Với giá trị `False` thì ta biết được đó là dữ liệu của hệ thống, còn với giá trị `True` thì là dữ liệu do chúng ta thêm vào
- **Output:** cung cấp cho ta một file với dữ liệu thời gian liền mạch

In [130]:
air_visit.index = pd.to_datetime(air_visit['visit_date'])
air_visit = air_visit.groupby('air_store_id').apply(lambda data: data['visitors'].resample('1d').sum(), include_groups=False).reset_index()
air_visit['visit_date'] = air_visit['visit_date'].dt.strftime('%Y-%m-%d') #chuyen dang YYYY-mm-dd
air_visit.replace(0, np.nan, inplace=True)
air_visit['was_nil'] = air_visit['visitors'].isnull()
air_visit.fillna({'visitors': 0}, inplace = True) #ignore

Dữ liệu file `air_visit_data.csv` sau khi xử lí là:

In [131]:

air_visit.head(10)

,air_store_id,visit_date,visitors,was_nil
0,air_00a91d42b08b08d9,2016-07-01,35.0,False
1,air_00a91d42b08b08d9,2016-07-02,9.0,False
2,air_00a91d42b08b08d9,2016-07-03,0.0,True
3,air_00a91d42b08b08d9,2016-07-04,20.0,False
4,air_00a91d42b08b08d9,2016-07-05,25.0,False
5,air_00a91d42b08b08d9,2016-07-06,29.0,False
6,air_00a91d42b08b08d9,2016-07-07,34.0,False
7,air_00a91d42b08b08d9,2016-07-08,42.0,False
8,air_00a91d42b08b08d9,2016-07-09,11.0,False
9,air_00a91d42b08b08d9,2016-07-10,0.0,True


#### 2.1.2 `File date_info.csv`

In [132]:
date_info = pd.read_csv('data/date_info.csv')
date_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   calendar_date  517 non-null    object
 1   day_of_week    517 non-null    object
 2   holiday_flg    517 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 12.2+ KB


- Thông qua việc xem dữ liệu file `date_info.csv` gồm 3 cột `calendar_date`, `day_of_week` và `holiday_flag`
- Ý nghĩa của các cột dữ liệu: 
    - **Với cột dữ liệu `calendar_date`:**
        - Đây là cột dữ liệu cung cấp thời gian 
        - Trạng thái: cột chứa 517 dòng dữ liệu, và không trống
        - Kiểu dữ liệu: `calendar_date` có kiểu dữ liệu là object
    - **Với cột dữ liệu `day_of_week`:**
        - Đây là cột dữ liệu cung cấp cho chúng ta các thứ trong tuần
        - Trạng thái: cột chứa 517 dòng dữ liệu, và không trống
        - Kiểu dữ liệu: `day_of_week` có kiểu dữ liệu là object
    - **Với cột dữ liệu `holiday_flg`:**
        - Đây là cột dữ liệu cung cấp nhãn về ngày đó có phải là ngày lễ hay không. Với giá trị là 1 thì đây là ngày lễ, ngược lại giá trị 0 thì không phải là ngày lễ
        - Trạng thái: cột chứa 517 dòng dữ liệu, và không trống
        - Kiểu dữ liệu: `holiday_flg` có kiểu dữ liệu là int64

In [133]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


- **Ý tưởng để xử lí file date_info.csv:**
    - Đổi tên 2 cột `calendar_date` và `holiday_flg` thành `visit_date` và `is_holiday` để có thể tiện dùng
    - Tạo thêm 2 cột mới: nhóm tạo thêm `prev_holiday` và `after_holiday` để xét ngày trước và sau của ngày đang xét là ngày lễ, giá trị của 2 cột đều giống với cột `is_holiday`
- **Ouput:** ta sẽ được file với dữ liệu cho biết rằng ngày đó có trong kì nghỉ dài hạn hay không

In [134]:
date_info.rename(columns = {'calendar_date' : 'visit_date', 'holiday_flg' : 'is_holiday'}, inplace = True)
date_info['prev_holiday'] = date_info['is_holiday'].shift().fillna(0)
date_info['after_holiday'] = date_info['is_holiday'].shift(-1).fillna(1)

Đây là file `date_info.csv` sau khi được xử lí:

In [135]:
date_info.head()

,visit_date,day_of_week,is_holiday,prev_holiday,after_holiday
0,2016-01-01,Friday,1,0.0,1.0
1,2016-01-02,Saturday,1,1.0,1.0
2,2016-01-03,Sunday,1,1.0,0.0
3,2016-01-04,Monday,0,1.0,0.0
4,2016-01-05,Tuesday,0,0.0,0.0


#### 2.1.3 `File air_store_info.csv`

Với file `air_store_info.csv`, nhóm đã sử dụng bảng đã được file đã được tiền xử lí sẵn nằm trong file dữ liệu thời tiết, lí do lựa chọn là vì file có sẵn dữ liệu trạm thời tiết trong đó, rất tiện cho việc join bảng sau này

In [136]:
path3 = 'weather/air_store_info_with_nearest_active_station.csv'
air_store_info = pd.read_csv(path3)
air_store_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   air_store_id          829 non-null    object 
 1   air_genre_name        829 non-null    object 
 2   air_area_name         829 non-null    object 
 3   latitude              829 non-null    float64
 4   longitude             829 non-null    float64
 5   latitude_str          829 non-null    object 
 6   longitude_str         829 non-null    object 
 7   station_id            829 non-null    object 
 8   station_latitude      829 non-null    float64
 9   station_longitude     829 non-null    float64
 10  station_vincenty      829 non-null    float64
 11  station_great_circle  829 non-null    float64
dtypes: float64(6), object(6)
memory usage: 77.8+ KB


- Thông qua việc đọc file `air_store_info_with_nearest_active_station.csv`, ta thấy được file có 11 cột như sau `air_store_id`, `air_genre_name`, `air_area_name`, `latitude`, `longitude`, `latitude_str`, `longitude_str`, `station_id`, `station_latitude`, `station_longitude`, `station_vincenty`, `station_great_circle`
- Ý nghĩa của các cột dữ liệu:
    - Với các cột `air_genre_name`, `air_area_name`:
        - Đây là các cột dữ liệu thể hiện loại hình, địa điểm của các nhà hàng
        - Trạng thái: các cột có 829 dòng dữ liệu, và không trống
        - Kiểu dữ liệu: đều mang kiểu dữ liệu object
    - Với các cột `latitude`, `longitude`, `station_latitude`, `station_longitude`, `station_vincenty` và `station_great_circle`:
        - Đây là các cột dữ liệu `latitude`, `longitude` thể hiện kinh độ, vĩ độ của các cửa hàng, `station_latitude`, `station_longitude`, `station_vincenty` và `station_great_circle` thể hiện kinh độ, vĩ độ của các trạm thời tiết, `station_vincenty`, `station_great_circle` thể hiện khoảng cách từ nhà hàng đến trạm thời tiết gần đó theo ước lượng và tính chính xác
        - Trạng thái: các cột đều có 829 dòng dữ liệu, và không trống
        - Kiểu dữ liệu: đều mang kiểu dữ liệu float64
    - Với các cột `langitude_str`, `longitude_str`, `station_id`:
        - Đây là các cột thể hiện kinh độ, vĩ độ dưới dạng string và id của trạm thời tiết gần đó
        - Trạng thái: các cột đều có 829 dòng dữ liệu, và không trống
        - Kiểu dữ liệu: đều mang kiểu dữ liệu object

In [137]:
air_store_info.head(10)

,air_store_id,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,"""35.6580681""","""139.7515992""",tokyo__tokyo-kana__tonokyo,35.691667,139.750000,3.730672,3.739835
5,air_99c3eae84130c1cb,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,"""35.6580681""","""139.7515992""",tokyo__tokyo-kana__tonokyo,35.691667,139.750000,3.730672,3.739835
6,air_f183a514cb8ff4fa,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,"""35.6580681""","""139.7515992""",tokyo__tokyo-kana__tonokyo,35.691667,139.750000,3.730672,3.739835
7,air_6b9fa44a9cf504a1,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,"""35.6580681""","""139.7515992""",tokyo__tokyo-kana__tonokyo,35.691667,139.750000,3.730672,3.739835
8,air_0919d54f0c9a24b8,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,"""35.6580681""","""139.7515992""",tokyo__tokyo-kana__tonokyo,35.691667,139.750000,3.730672,3.739835
9,air_2c6c79d597e48096,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,"""35.6580681""","""139.7515992""",tokyo__tokyo-kana__tonokyo,35.691667,139.750000,3.730672,3.739835


#### 2.1.4 `File submission.csv`

In [138]:
submission = pd.read_csv('data/sample_submission.csv')
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32019 entries, 0 to 32018
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        32019 non-null  object
 1   visitors  32019 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 500.4+ KB


- File `submission` là file kết quả chúng ta sẽ nộp trên Kaggle còn được xem là file test sau này
- Thông qua việc đọc`submission.csv`, file chứa 2 cột là `id`, `visitors`
- Ý nghĩa của các cột dữ liệu:
    -  Với cột dữ liệu `id`:
        - Đây là từ khóa bao gồm id của cửa hàng + ngày có khách, thông qua từ khóa sẽ dự đoán ra số khách tương ứng
        - Trạng thái: cột id có 32019 dòng dữ liệu, và không trống
        - Kiểu dữ liệu: cột `id` có kiểu dữ liệu object

- **Ý tưởng xử lí file `submission`**:
    - Lấy cột id của cửa hàng, ngày có khách: Thông qua str.slice() để trích xuất từ cột `id`, với 21 kí tự đầu là id của cửa hàng, từ kí tự thứ 21 trở đi là ngày có khách
    - Đánh dấu tập test: tạo thêm 1 cột `is_test` với giá trị là True
    - Đánh số theo index cho từng dòng

In [139]:

submission['air_store_id'] = submission['id'].str.slice(0,20)
submission['visit_date'] = submission['id'].str.slice(21)
submission['visit_date'] = pd.to_datetime(submission['visit_date']).dt.strftime('%Y-%m-%d')
submission['visitors'] = np.nan #để bỏ qua cột này
submission['is_test'] = True
submission['is_number'] = submission.index

submission.head()


,id,visitors,air_store_id,visit_date,is_test,is_number
0,air_00a91d42b08b08d9_2017-04-23,NaN,air_00a91d42b08b08d9,2017-04-23,True,0
1,air_00a91d42b08b08d9_2017-04-24,NaN,air_00a91d42b08b08d9,2017-04-24,True,1
2,air_00a91d42b08b08d9_2017-04-25,NaN,air_00a91d42b08b08d9,2017-04-25,True,2
3,air_00a91d42b08b08d9_2017-04-26,NaN,air_00a91d42b08b08d9,2017-04-26,True,3
4,air_00a91d42b08b08d9_2017-04-27,NaN,air_00a91d42b08b08d9,2017-04-27,True,4


#### 2.1.5 Merge dataset

Mục đích merge để có tập dữ liệu đầy đủ và hoàn thiện hơn cho mô hình
- **Ý tưởng xử lí sau khi merge:**
    - Thay những giá trị Nan trong cột `is_test` bằng False, giải thích về điều này là vì sau khi ghép 2 bảng lại thì những dữ liệu trong file test thì giá trị cột `is_test` là True trong khi những cột khác lại là Nan nên ta mới có cách điều chỉnh như trên
    - Chuyển kiểu dữ liệu của cột `visitors` nhằm giữ lại các Nan là các giá trị của cột `visitors` nằm trong tập Test

In [140]:
data = pd.concat((air_visit, submission.drop(['id'], axis = 'columns')))
data['is_test']  = data['is_test'].fillna(False).astype('bool') #vì cột khi ép có type là object, dùng astype để chuyển sang boolean
data = pd.merge(left = data, right = date_info, on = 'visit_date', how = 'left')
data = pd.merge(left = data, right = air_store_info, on = 'air_store_id', how = 'left')
data['visitors'] = data['visitors'].astype(float)
data.head()

,air_store_id,visit_date,visitors,was_nil,is_test,is_number,day_of_week,is_holiday,prev_holiday,after_holiday,...,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle
0,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
1,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
2,air_00a91d42b08b08d9,2016-07-03,0.0,True,False,NaN,Sunday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
3,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
4,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906


### 2.2. Dữ liệu thời tiết

- **Input:** ta sẽ có dữ liệu 1663 trạm đo thời tiết
- **Output:** ta sẽ có file dữ liệu bao gồm tất cả 1663 trạm đo 

In [141]:
import glob
from pathlib import Path
weather_df = []
for path in glob.glob('weather/1-1-16_5-31-17_Weather/1-1-16_5-31-17_Weather/*.csv'):
    weather = pd.read_csv(path)
    weather['station_id'] = Path(path).stem
    weather_df.append(weather)


weather = pd.concat(weather_df, axis='rows')
weather.rename(columns={'calendar_date': 'visit_date'}, inplace=True)


In [142]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 859771 entries, 0 to 516
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   visit_date          859771 non-null  object 
 1   avg_temperature     480234 non-null  float64
 2   high_temperature    480189 non-null  float64
 3   low_temperature     480189 non-null  float64
 4   precipitation       640279 non-null  float64
 5   hours_sunlight      433473 non-null  float64
 6   solar_radiation     24803 non-null   float64
 7   deepest_snowfall    123827 non-null  float64
 8   total_snowfall      120157 non-null  float64
 9   avg_wind_speed      473007 non-null  float64
 10  avg_vapor_pressure  80383 non-null   float64
 11  avg_local_pressure  81048 non-null   float64
 12  avg_humidity        80384 non-null   float64
 13  avg_sea_pressure    78054 non-null   float64
 14  cloud_cover         30412 non-null   float64
 15  station_id          859771 non-null  objec

- Thông qua việc đọc file tất cả các file .csv trong weather, ta thấy được có 16 cột bao gồm `visit_date`, `avg_temperature`, `high_temperature`, `low_temperature`, `precipitation`, `hours_sunlight`, `solar_radiation`, `deepest_snowfall`, `total_snowfall`, `avg_wind_speed`, `avg_vapor_pressure`, `avg_local_pressure`, `avg_humidity`, `avg_sea_pressure`, `cloud_cover` và `station_id`:
- Nhìn qua tổng quan thì file có rất nhiều cột bị null, chỉ có cột lượng mưa `precipitation` và nhiệt độ trung bình `avg_temperature` thì ổn hơn và có vẻ dùng được
- Giải thích vì lí do chỉ chọn lượng mưa và nhiệt độ trung bình là vì:
    - `high_temperature` và `low_temperature` chỉ cho ta thấy được nhiệt độ cao nhất hoặc nhiệt độ thấp nhất trong ngày nhưng thứ ta cần lại là cái tổng quan hơn nên `avg_temperature` là lựa chọn hoàn hảo hơn
    - Còn lại đa số dữ liệu thiếu quá nhiều nên không thể dùng được, chỉ có lượng mưa thì đầy đủ nhất nên chúng ta sẽ chọn lượng mưa
- Còn về việc vì sao lại có việc dữ liệu lại thiếu nhiều như vậy, đó là bởi vì:
    - Một số trạm sẽ đo các đặc trưng thời tiết ở một vùng nào đó, ví dụ vùng nắng nóng thì không thể nào có tuyết rơi được và ngược lại
    - Một số trạm sẽ đo nhiều đặc trưng nhưng cũng có trạm chỉ đo một vài đặc trưng nào đó như lượng mưa, nhiệt độ
    $\rightarrow$ Điều đó dẫn đến việc dữ liệu bị thiếu như vậy

#### 2.2.2 Trích xuất dữ liệu thời tiết

- **Một số thao tác trước khi lấy những đặc trưng cần thiết:**
    - Tính nhiệt độ trung bình và lượng mưa trung bình theo từng ngày trên một dataframe means
    - Ghép means với weather và thay thế các vị trí Nan bằng giá trị nhiệt độ trung bình và lượng mưa trubg bình theo từng ngày ta đã tính trước đó

In [143]:
means = weather.groupby('visit_date')[['avg_temperature', 'precipitation']].mean().reset_index()
means.rename(columns={'avg_temperature': 'global_avg_temperature', 'precipitation': 'global_precipitation'}, inplace=True)
weather = pd.merge(left=weather, right=means, on='visit_date', how='left')
weather['avg_temperature'].fillna(weather['global_avg_temperature'], inplace=True)
weather['precipitation'].fillna(weather['global_precipitation'], inplace=True)

weather[['visit_date', 'avg_temperature', 'precipitation']].head()

,visit_date,avg_temperature,precipitation
0,2016-01-01,6.0,0.0
1,2016-01-02,4.7,0.0
2,2016-01-03,7.0,0.0
3,2016-01-04,8.8,0.0
4,2016-01-05,8.9,0.0


In [144]:
data = pd.merge(
    left=data, 
    right=weather[['station_id', 'visit_date', 'avg_temperature', 'precipitation']], 
    on=['station_id', 'visit_date'], 
    how='left'
)

In [145]:
data['visit_date'] = pd.to_datetime(data['visit_date'])
data = data.sort_values(['air_store_id', 'visit_date'])
data = data.set_index('visit_date', drop=False)

data.head()

,air_store_id,visit_date,visitors,was_nil,is_test,is_number,day_of_week,is_holiday,prev_holiday,after_holiday,...,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle,avg_temperature,precipitation
visit_date,,,,,,,,,,,,,,,,,,,,,
2016-07-01,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,...,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,25.6,0.70462
2016-07-02,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,...,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,27.0,0.00000
2016-07-03,air_00a91d42b08b08d9,2016-07-03,0.0,True,False,NaN,Sunday,0,0.0,0.0,...,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,29.2,7.29701
2016-07-04,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,...,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,27.8,1.50000
2016-07-05,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,...,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,21.7,0.00000


## 2.3. Tiền xử lí dữ liệu

### 2.3.1 Xử lí giá trị ngoại lai

- **Phương pháp:** Sử dụng phương pháp Z-score với ngưỡng k = 2.4
    - Phương pháp Z-score có công thức $Z = \frac{x - \bar{x}}{\sigma}$, với Z là một giá trị thống kê kiểm tra độ lệch của từng quan sát, nếu $|Z|$ lớn hơn ngưỡng k thì quan sát đó được xem như 1 outlier
    - Với việc tìm outlier cho cột `visitors` thì đồng nghĩa với việc tìm ra những ngày có số lượng khách tăng đột biến, có khả năng làm sai lệch quá trình học của mô hình, ví dụ như những ngày Tết, hoặc ngày lễ, ... Việc tăng cao sẽ làm cho mô hình đánh giá quá cao lượng khách trung bình và ở đây cột `visitors` không có giá trị âm nên ta chỉ cần kiểm tra Z lớn hơn ngưỡng k
- **Việc xử lí outlier:** Sau khi tìm được outlier thì chúng ta sẽ đánh dấu theo giá trị True/False bằng cột `outlier`, sau đó sẽ tạo thêm cột `replace_visitor` để thay những giá trị outlier bằng max của giá trị không outlier
- Và ta tạo thêm một cột `replace_visitor_log1` để lấy log transform của cột `replaced_visitors` để nhằm giảm ảnh hưởng của các giá trị quá lớn, làm cho phân phối trở nên cân bằng, hạn chế tác động của outlier. Chúng ta có thể tham khảo thêm [tại đây](https://www.geeksforgeeks.org/data-science/log-transformation/)

In [146]:
def findOutlier(data, k=2.4):
    z = (data - data.mean()) / data.std()
    return z > k

def replace_outlier(data):
    outlier = findOutlier(data)
    max = data[~outlier].max()
    data[outlier] = max
    return data

store = data.groupby('air_store_id')
data['outlier'] = store.apply(lambda g: findOutlier(g['visitors'])).values
data['replace_visitors'] = store.apply(lambda g: replace_outlier(g['visitors'])).values
data['replace_visitor_log1'] = np.log1p(data['replace_visitors'])
data.head()

,air_store_id,visit_date,visitors,was_nil,is_test,is_number,day_of_week,is_holiday,prev_holiday,after_holiday,...,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle,avg_temperature,precipitation,outlier,replace_visitors,replace_visitor_log1
visit_date,,,,,,,,,,,,,,,,,,,,,
2016-07-01,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,...,tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,25.6,0.70462,False,35.0,3.583519
2016-07-02,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,...,tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,27.0,0.00000,False,9.0,2.302585
2016-07-03,air_00a91d42b08b08d9,2016-07-03,0.0,True,False,NaN,Sunday,0,0.0,0.0,...,tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,29.2,7.29701,False,0.0,0.000000
2016-07-04,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,...,tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,27.8,1.50000,False,20.0,3.044522
2016-07-05,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,...,tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,21.7,0.00000,False,25.0,3.258097


### 2.3.2 Trích xuất các đặc trưng khác

- Ở đây chúng ta sẽ lấy ra ngày cuối tuần và các ngày trong tháng, vì sao chúng ta lại trích xuất các đặc trưng này? Bởi vì nó sẽ phản ánh hành vi của khách hàng theo thời gian. Chẳng hạn như cuối tuần, khách hàng sẽ đi chơi nhiều, có nhu cầu ăn uống nhiều hơn nên số lượng khách sẽ tăng so với trong tuần, hoặc khi đầu tháng họ sẽ được nhận lương thì mức chi tiêu của họ sẽ khác cuối tháng. Vì vậy khi trích xuất đặc trưng `weekend` và `day_of_month`, mô hình sẽ học được quy luật và hành vi của khách hàng thông qua từng thời điểm trong tuần và trong tháng

In [147]:
data['weekend'] = data['day_of_week'].isin(['Saturday', 'Sunday']).astype(int)
data['day_of_month'] = data['visit_date'].dt.day

### 2.3.3 Trọng số theo cấp số nhân cho số lượng khách hàng

- EWM(Exponential Weighted Mean) là một cách tính trung bình với dữ liệu càng gần với hiện tại thì càng quan trọng, nó được biểu diễn qua công thức:
$$\text{EWM}_t = \alpha x_{t-1} + (1 - \alpha)\text{EWM}_{t-1}$$
- Với việc giảm dần theo hàm mũ cho thấy rằng khi dữ liệu càng về quá khứ thì dữ liệu đó không giúp gì nhiều cho dữ liệu hiện tại giúp nắm bắt được xu hướng ngắn hạn và phản ánh được hành vi gần đây của khách hàng
- Ở đoạn code có một chỗ đó là **return data.shift().ewm(alpha = alpha, adjust = adjust).mean()**, mục đích của việc dùng shift() ở đây là để có thể tránh việc data leakage, nếu mà không có shift() thì chắc chắn mô hình sẽ biết được đáp án và thay vì dự đoán, mô hình sẽ "học lõm" dẫn đến hiệu suất huấn luyện cao một cách bất thường, nhưng lại suy giảm nghiêm trọng khi áp dụng ở tập validation hoặc dữ liệu tương lai

In [148]:

from scipy import optimize
def cal_ewm(data, alpha, adjust = False):
    return data.shift().ewm(alpha = alpha, adjust = adjust).mean()
def find_best_alpha(data, adjust = False, eps = 10e-5):
    def f(alpha):
        ewm_estimate = cal_ewm(data, alpha = min(max(alpha, 0), 1), adjust= adjust)
        error = np.mean((data - ewm_estimate)**2)
        return error
    best = optimize.differential_evolution(func = f, bounds = [(0 + eps, 1 - eps)])
    return cal_ewm(data, alpha = best['x'][0], adjust = adjust)

group = data.groupby(['air_store_id', 'day_of_week']).apply(lambda g: find_best_alpha(g['replace_visitors']))
data['optimize_week_visitor'] = group.sort_index(level=['air_store_id', 'visit_date']).values

group = data.groupby(['air_store_id', 'day_of_week']).apply(lambda g: find_best_alpha(g['replace_visitor_log1']))
data['optimize_week_visitor_log1'] = group.sort_index(level = ['air_store_id', 'visit_date']).values

group = data.groupby(['air_store_id', 'weekend']).apply(lambda g: find_best_alpha(g['replace_visitors']))
data['optimize_weekend_visitor']= group.sort_index(level= ['air_store_id', 'visit_date']).values

group = data.groupby(['air_store_id', 'weekend']).apply(lambda g: find_best_alpha(g['replace_visitor_log1']))
data['optimize_weekend_visitor_log1p'] = group.sort_index(level = ['air_store_id', 'visit_date']).values


data.head()

,air_store_id,visit_date,visitors,was_nil,is_test,is_number,day_of_week,is_holiday,prev_holiday,after_holiday,...,precipitation,outlier,replace_visitors,replace_visitor_log1,weekend,day_of_month,optimize_week_visitor,optimize_week_visitor_log1,optimize_weekend_visitor,optimize_weekend_visitor_log1p
visit_date,,,,,,,,,,,,,,,,,,,,,
2016-07-01,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,...,0.70462,False,35.0,3.583519,0,1,NaN,NaN,NaN,NaN
2016-07-02,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,...,0.00000,False,9.0,2.302585,1,2,NaN,NaN,NaN,NaN
2016-07-03,air_00a91d42b08b08d9,2016-07-03,0.0,True,False,NaN,Sunday,0,0.0,0.0,...,7.29701,False,0.0,0.000000,1,3,NaN,NaN,9.000000,2.302585
2016-07-04,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,...,1.50000,False,20.0,3.044522,0,4,NaN,NaN,35.000000,3.583519
2016-07-05,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,...,0.00000,False,25.0,3.258097,0,5,NaN,NaN,33.565202,3.429266


#### 2.3.4. Tính các đặc trưng khác

Hàm rollStatistic được sử dụng để tạo ra các đặc trưng thống kê dựa trên lịch sử số lượng khách của từng nhà hàng. Các thống kê bao gồm trung bình, độ biến động và xu hướng theo thời gian, được tính riêng cho từng bối cảnh như thứ trong tuần và cuối tuần. Việc sử dụng shift() đảm bảo rằng các đặc trưng chỉ dựa trên dữ liệu quá khứ, tránh hiện tượng data leakage. Những đặc trưng này giúp mô hình học được hành vi và quy luật của khách hàng theo thời gian.

In [149]:

def rollStatistic(data, col, group_by):
    data.index.name = None
    data.sort_values(group_by + ['visit_date'], inplace = True)
    data_group = data.groupby(group_by, sort = False)
    stats = {
        'mean' : [],    #muc khach trung binh
        'median' : [],  #xu huong on dinh
        'std' : [],     #muc do on dinh
        'count' : [],   #do dai lich su
        'max' : [],     #muc dong nhat
        'min' : []      #muc thap nhat
    }
    alpha_exp = [0.1, 0.25, 0.3, 0.5, 0.75]
    stats.update({'ewm_{}_mean'.format(alpha) : [] for alpha in alpha_exp})
    for _, group in data_group:
        shift = group[col].shift()
        rolling = shift.rolling(window = len(group), min_periods = 1)
        stats['mean'].extend(rolling.mean())
        stats['median'].extend(rolling.median())
        stats['std'].extend(rolling.std())
        stats['count'].extend(rolling.count())
        stats['max'].extend(rolling.max())
        stats['min'].extend(rolling.min())
        for alpha in alpha_exp:
            ewm_alpha = shift.ewm(alpha = alpha, adjust = False)
            stats['ewm_{}_mean'.format(alpha)].extend(ewm_alpha.mean())
    merge_name = '_&_'.join(group_by)
    for stats_name, value in stats.items():
        data['{}_{}_{}'.format(col, stats_name, merge_name)] = value

rollStatistic(data, col = 'replace_visitors', group_by = ['air_store_id', 'day_of_week'])
rollStatistic(data, col = 'replace_visitors', group_by = ['air_store_id', 'weekend'])
rollStatistic(data, col = 'replace_visitors', group_by = ['air_store_id'])

rollStatistic(data, col = 'replace_visitor_log1', group_by = ['air_store_id', 'day_of_week'])
rollStatistic(data, col = 'replace_visitor_log1', group_by = ['air_store_id', 'weekend'])
rollStatistic(data, col = 'replace_visitor_log1', group_by = ['air_store_id'])
data.sort_values(['air_store_id', 'visit_date'])
    

,air_store_id,visit_date,visitors,was_nil,is_test,is_number,day_of_week,is_holiday,prev_holiday,after_holiday,...,replace_visitor_log1_median_air_store_id,replace_visitor_log1_std_air_store_id,replace_visitor_log1_count_air_store_id,replace_visitor_log1_max_air_store_id,replace_visitor_log1_min_air_store_id,replace_visitor_log1_ewm_0.1_mean_air_store_id,replace_visitor_log1_ewm_0.25_mean_air_store_id,replace_visitor_log1_ewm_0.3_mean_air_store_id,replace_visitor_log1_ewm_0.5_mean_air_store_id,replace_visitor_log1_ewm_0.75_mean_air_store_id
2016-07-01,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-02,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,...,3.583519,NaN,1.0,3.583519,3.583519,3.583519,3.583519,3.583519,3.583519,3.583519
2016-07-03,air_00a91d42b08b08d9,2016-07-03,0.0,True,False,NaN,Sunday,0,0.0,0.0,...,2.943052,0.905757,2.0,3.583519,2.302585,3.455426,3.263285,3.199239,2.943052,2.622819
2016-07-04,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,...,2.302585,1.815870,3.0,3.583519,0.000000,3.109883,2.447464,2.239467,1.471526,0.655705
2016-07-05,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,...,2.673554,1.578354,4.0,3.583519,0.000000,3.103347,2.596729,2.480984,2.258024,2.447318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-05-27,air_fff68b929994bfbd,2017-05-27,NaN,NaN,True,32014.0,Saturday,0,0.0,0.0,...,1.609438,0.688948,296.0,2.564949,0.000000,1.596278,1.568845,1.570479,1.616823,1.717177
2017-05-28,air_fff68b929994bfbd,2017-05-28,NaN,NaN,True,32015.0,Sunday,0,0.0,0.0,...,1.609438,0.688948,296.0,2.564949,0.000000,1.596278,1.568845,1.570479,1.616823,1.717177
2017-05-29,air_fff68b929994bfbd,2017-05-29,NaN,NaN,True,32016.0,Monday,0,0.0,0.0,...,1.609438,0.688948,296.0,2.564949,0.000000,1.596278,1.568845,1.570479,1.616823,1.717177
2017-05-30,air_fff68b929994bfbd,2017-05-30,NaN,NaN,True,32017.0,Tuesday,0,0.0,0.0,...,1.609438,0.688948,296.0,2.564949,0.000000,1.596278,1.568845,1.570479,1.616823,1.717177


## 3.Huấn luyện mô hình

### 3.1. Mã hóa dữ liệu bằng one-hot encoding

- Vì sao chúng ta lại dùng one-hot encoding? Thực ra thì do mô hình không hiểu được chữ như Monday, Tuesday,... nhưng nếu ta mã hóa bằng số (Monday = 1, Tuesday = 2, ...) thì mô hình lại hiểu sai rằng những thứ đó là có mức thứ tự, trong đó những thứ có thứ tự cao được xử lí như một mức cao hơn dù không có quan hệ thứ bậc
- Việc dùng one-hot encoding giúp cho các thứ trong tuần trở nên độc lập, với việc đánh giá trị 1 cho các vị trí của thứ hoặc loại hình nhà hàng được đưa ra và 0 cho các vị trí mà không phải là thứ hoặc loại hình nhà hàng đó

In [150]:
data = pd.get_dummies(data, columns=['day_of_week', 'air_genre_name'])
data.head()

,air_store_id,visit_date,visitors,was_nil,is_test,is_number,is_holiday,prev_holiday,after_holiday,air_area_name,...,air_genre_name_Dining bar,air_genre_name_International cuisine,air_genre_name_Italian/French,air_genre_name_Izakaya,air_genre_name_Japanese food,air_genre_name_Karaoke/Party,air_genre_name_Okonomiyaki/Monja/Teppanyaki,air_genre_name_Other,air_genre_name_Western food,air_genre_name_Yakiniku/Korean food
2016-07-01,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,0,0.0,0.0,Tōkyō-to Chiyoda-ku Kudanminami,...,False,False,True,False,False,False,False,False,False,False
2016-07-02,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,0,0.0,0.0,Tōkyō-to Chiyoda-ku Kudanminami,...,False,False,True,False,False,False,False,False,False,False
2016-07-03,air_00a91d42b08b08d9,2016-07-03,0.0,True,False,NaN,0,0.0,0.0,Tōkyō-to Chiyoda-ku Kudanminami,...,False,False,True,False,False,False,False,False,False,False
2016-07-04,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,0,0.0,0.0,Tōkyō-to Chiyoda-ku Kudanminami,...,False,False,True,False,False,False,False,False,False,False
2016-07-05,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,0,0.0,0.0,Tōkyō-to Chiyoda-ku Kudanminami,...,False,False,True,False,False,False,False,False,False,False


### 3.2. Chuẩn bị dữ liệu train, test:
- Tạo cột visitor_log1p bằng cách lấy log transform của cột dữ liệu `visitors`
- Với dữ liệu train:
    Nhóm sẽ loại đi các dữ liệu thuộc tập test `data['is_test']`, những dữ liệu bị outlier và những giá trị NULLL. Sau đó loại đi những cột không cần thiết khác. Chúng ta sẽ chia tập đầu vào X: là dữ liệu train bỏ đi cột `visitor_log1p`. Còn tập mục tiêu Y: ta chỉ lấy cột `visitor_log1p`

- Với dữ liệu test:
    Ta lấy những giá trị mà `is_test` hiển thị là True. Cũng loại đi những cột không cần thiết, và X_test cũng loại đi cột `visitor_log1p`

In [151]:
data['visitor_log1p'] = np.log1p(data['visitors'])
train = data[(data['is_test'] == False) & (data['outlier'] == False) & (data['was_nil'] == False)]
test = data[data['is_test'] == True].sort_values('is_number')
#liet ke cac cot de xoa
col_drop = ['air_store_id', 'visitors', 'is_test', 'outlier', 'visit_date', 'was_nil', 'is_number',
            'replace_visitors', 'air_area_name', 'station_id', 'station_latitude', 'station_longitude', 'station_vincenty',
            'station_great_circle', 'replace_visitor_log1', 'latitude_str', 'longitude_str']

train = train.drop(col_drop, axis = 'columns')
train = train.dropna()
X_train = train.drop('visitor_log1p', axis = 'columns')
Y_train = train['visitor_log1p']

test = test.drop(col_drop, axis = 'columns')
X_test = test.drop('visitor_log1p', axis = 'columns')
X_test.head()

,is_holiday,prev_holiday,after_holiday,latitude,longitude,avg_temperature,precipitation,weekend,day_of_month,optimize_week_visitor,...,air_genre_name_Dining bar,air_genre_name_International cuisine,air_genre_name_Italian/French,air_genre_name_Izakaya,air_genre_name_Japanese food,air_genre_name_Karaoke/Party,air_genre_name_Okonomiyaki/Monja/Teppanyaki,air_genre_name_Other,air_genre_name_Western food,air_genre_name_Yakiniku/Korean food
2017-04-23,0,0.0,0.0,35.694003,139.753595,13.6,0.500000,1,23,0.000200,...,False,False,True,False,False,False,False,False,False,False
2017-04-24,0,0.0,0.0,35.694003,139.753595,14.5,0.400254,0,24,19.994625,...,False,False,True,False,False,False,False,False,False,False
2017-04-25,0,0.0,0.0,35.694003,139.753595,16.0,2.007692,0,25,29.836148,...,False,False,True,False,False,False,False,False,False,False
2017-04-26,0,0.0,0.0,35.694003,139.753595,16.9,0.000000,0,26,27.973552,...,False,False,True,False,False,False,False,False,False,False
2017-04-27,0,0.0,0.0,35.694003,139.753595,14.7,3.500000,0,27,33.489572,...,False,False,True,False,False,False,False,False,False,False


### 3.3.LightGBM + KFold

- LightGBM: là một framework học tăng cường theo gradient, tận dụng các thuật toán học dựa trên cây, đặc biệt là decision tree đã tối ưu.
- Đặc điểm và tối ưu hóa nổi bật của LightGBM là:
    - Phát triển cây theo lá: Khác với những phương pháp triển khai khác thường mở rộng cây theo từng tầng, thì LightGBM sẽ chọn là có delta loss (độ giảm tổn thất) lớn nhất để chia
    - Thuật toán dựa trên histogram: LightGBM dùng thuật toán dựa trên histogram để tìm điểm chia tối ưu
    - Tiết kiệm bộ nhớ
    - Hỗ trợ học song song với phân tán

<div align="center">
  <img src="leaf-wise.webp" alt="Ảnh minh họa" width="500">
</div>

- K-Fold là phương pháp đánh giá chéo dữ liệu bằng cách chia nhỏ dữ liệu train và biến chúng thành từng bộ train và test data. Số lượng fold càng lớn thì dữ liệu chia càng lớn. Điều đó đảm bảo việc đánh giá dữ liệu khách quan hơn.
- Trong bài toán này, ta sử dụng 6 fold như một phương pháp cải thiện sự thiếu hụt về dữ liệu của bài toán, đồng thời đảm bảo được hiện tượng overfitting không xảy ra do tỉ lệ tập val và train.

In [152]:
np.random.seed(42)

# --- model ---
model = lgb.LGBMRegressor(
    objective='regression',
    max_depth=5,
    num_leaves=31,
    learning_rate=0.01,
    n_estimators=30000,
    min_child_samples=80,
    subsample=0.8,
    colsample_bytree=1,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=np.random.randint(10e6),
    verbosity=-1
)

n_splits = 6
cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)

val_scores = [0] * n_splits

sub = submission['id'].to_frame()
sub['visitors'] = 0

feature_importances = pd.DataFrame(index=X_train.columns)

for i, (fit_idx, val_idx) in enumerate(cv.split(X_train, Y_train)):

    X_fit = X_train.iloc[fit_idx]
    y_fit = Y_train.iloc[fit_idx]
    X_val = X_train.iloc[val_idx]
    y_val = Y_train.iloc[val_idx]

    # --- tạo progress bar ---
    pbar = tqdm(total=model.n_estimators, desc=f'Fold {i+1}', ncols=100)

    def tqdm_callback(env):
        """Callback cho tqdm, gọi mỗi iteration"""
        pbar.update(1)

    # train model với callback
    model.fit(
        X_fit,
        y_fit,
        eval_set=[(X_val, y_val)],
        eval_names=['val'],
        eval_metric='l2',
        feature_name=X_fit.columns.tolist(),
        callbacks=[tqdm_callback]
    )

    pbar.close()

    val_scores[i] = np.sqrt(model.best_score_['val']['l2'])
    sub['visitors'] += model.predict(X_test, num_iteration=model.best_iteration_)
    feature_importances[i] = model.feature_importances_

# trung bình kết quả
sub['visitors'] /= n_splits
sub['visitors'] = np.expm1(sub['visitors'])

val_mean = np.mean(val_scores)
val_std = np.std(val_scores)

print('Local RMSLE: {:.5f} (±{:.5f})'.format(val_mean, val_std))

Fold 6: 100%|████████████████████████████████████████████████| 30000/30000 [03:37<00:00, 138.02it/s]


Local RMSLE: 0.48643 (±0.00203)


### 3.4.Xuất ra file kết quả Submission

In [153]:
sub.to_csv('submissionslgbm_{:.5f}_{:.5f}.csv'.format(val_mean, val_std), index=False)
fname = 'submissionslgbm_{:.5f}_{:.5f}.csv'.format(val_mean, val_std)
sub.to_csv(fname, index=False)
sub_check = pd.read_csv(fname)
sub_check['visitors'] = sub_check['visitors'].round().astype('int64')
sub_check.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,3
1,air_00a91d42b08b08d9_2017-04-24,25
2,air_00a91d42b08b08d9_2017-04-25,27
3,air_00a91d42b08b08d9_2017-04-26,30
4,air_00a91d42b08b08d9_2017-04-27,31


### 4. Kết luận và đề xuất cải tiến:
- Kết luận: 
    Báo cáo đã xây dựng một quy trình hoàn chỉnh cho bài toán dự đoán số lượng khách dựa trên dữ liệu lịch sử nhà hàng. Dữ liệu được tiền xử lý theo trục thời gian, xử lý outlier và biến đổi log nhằm giảm ảnh hưởng của giá trị bất thường và phù hợp với thước đo RMSLE. Mô hình LightGBM được sử dụng kết hợp với cross-validation để đánh giá hiệu năng một cách khách quan. Kết quả cho thấy mô hình cho sai số ổn định và có khả năng dự đoán tốt trên tập validation, đáp ứng yêu cầu của bài toán
- Đề xuất cải tiến
    Trích chọn dữ liệu thời tiết, ta thu thập thông tin về nhiệt độ trung bình trên toàn bộ dữ liệu. Nếu có thời gian nhóm sẽ nghiên cứu thu thập dữ liệu theo từng vùng cụ thể để nhằm tăng độ chính xác cho thông tin thời tiết tại một khu vực cụ thể. Nếu có thời gian thì nhóm sẽ tìm hiểu thêm các mô hình khác. 

### 5. Tài liệu tham khảo

1. https://www.kaggle.com/competitions/recruit-restaurant-visitor-forecasting/data
2. https://lightgbm.readthedocs.io/en/latest/index.html
3. https://www.geeksforgeeks.org/data-science/log-transformation/
4. https://github.com/MaxHalford/kaggle-recruit-restaurant/tree/master
5. https://www.w3schools.com/statistics/statistics_standard_normal_distribution.php
6. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
7. https://www.geeksforgeeks.org/data-science/log-transformation/